In [ ]:
"""
MODIFICADO DE cpv2010arg.py en https://github.com/abenassi/pyredatam/blob/1480c481feb0698d54b59c3c17e52661a8c793df/pyredatam/cpv2010arg.py
"""

import redapy

import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import import_ipynb
from Redapy_query import query_final

area = ["Provinci"]
var1 = ["Poblacio.C5P12"]
var2 = ["Vivienda.ENCAREA"]
selection = ["Provinci 0101","Provinci 0102"]
filter_a = "Distrito"

query = query_final(tipo="crosstab",variables1=var1,variables2=var2,area_break=area)

BASE_URL = "https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017DI&ITEM=PROGRED&lang=esp"
DRIVER_PATH = "C:\\Users\\Dieguinchi\\Desktop\\chromedriver.exe"

def make_query(query, url=BASE_URL): # hace consulta "query" a redatam a través de procesador estadístico online
    options = webdriver.ChromeOptions()#carga configuración del webdriver
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)# el driver debe estar en archivos del pc. puede descargarse de https://chromedriver.chromium.org/downloads
    driver.get(url) # abre pagina web de redatam
    query_input = driver.find_element_by_tag_name("textarea")# ubica linea de comandos
#     query_input.send_keys(query.decode("utf-8", "ignore"))
    query_input.send_keys(query) # escribe consulta en línea de comandos
    submit = driver.find_element_by_name("Submit") #Busca botón "Ejecutar"
    submit.click()# clickea en "Ejecutar" y ejecuta consulta
    WebDriverWait(driver, 999999).until(lambda driver: len(driver.find_elements_by_xpath("//*[contains(text(),'Descargar en formato Excel')]")) == int(len(var1)))#espera 999 segundos o a que se muestren todas las tablas solicitadas, es decir, debe mostrar Descargar en formato Excel la misma cantidad de veces que la cantidad de variables de la solicitud 
    html = driver.find_element_by_id("tab-output")# obtiene unicamente la tabla de resultados
    html = html.get_attribute('outerHTML')# obtiene el html de la tabla de resultados
    driver.close() # cierra navegador
    return html # devuelve html con tabla de resultados

def make_dataframe(html): # crea dataframe con resultados
    temp_table = pd.read_html(html) # lee todos los dataframes de la tabla de resultados
    return temp_table # devuelve lista con dataframes

merged = pd.concat(make_dataframe(make_query(query))) # concatena dataframes de la tabla de resultados en un solo dataframe
Excelwriter = pd.ExcelWriter("tablas_redatam2017_cruzada.xlsx",engine="xlsxwriter") #exporta en excel dataframe final
merged.to_excel(Excelwriter)
Excelwriter.save()

In [3]:
def clean_str(df, column=None):
    df=df.copy()
    dic_replace={"Á":"A","É":"E","Í":"I","Ó":"O","Ú":"U","-":"","\/":""}
    df[column]=(df[column].
                  str.upper().
                  str.strip().
                  replace(dic_replace, regex=True)
                 )
    return df
def list_ubigeo_index(df):
    ubigeo=(df.query('resp.notna() and resp.str.startswith("AREA")')
            ["resp"].
            copy()
           ) ## extrae los ubigeos de cada tabla
    list_index_d=ubigeo.index.values
    return ubigeo, list_index_d

def extrac_freq_ubigeo(df,i_start_var):
    
    ubigeo, list_index=list_ubigeo_index(df)
    index_table=list_index[i_start_var]
    
    # ubigeo_nom=ubigeo.loc[index_table:index_table+1].str.split("")[1][7:]
    for p,q in ubigeo.loc[index_table:index_table+1].items():
        ubigeo_nom=q[7:]
    ##
    i_start_u=df.loc[index_table+3:]
    i_end_u=i_start_u.query('resp.notna() and resp=="TOTAL"')[i_start_u.columns[0]].index[0]-1
    df_f=df.loc[i_start_u.index.values[0]:i_end_u].copy()
    df_f["ubigeo"]=str(ubigeo_nom)
    
    return df_f

def clean_columns_answers(df):
    df=df.copy()
    cols=df.iloc[0:1,0:].values.flatten().tolist()
    cols[-1:]=["ubigeo"]
    df.columns=cols
    df=df.iloc[1:].copy()
    return df

In [21]:
df=merged.iloc[5:].copy()
df=merged.copy()
df=df.iloc[:,1:].copy()

df=(df.
    rename({df.columns[0]:"resp"},axis=1).
    pipe(clean_str, column="resp")
   )

In [22]:
ubigeo, list_index_f=list_ubigeo_index(df)
ubigeos_des=[]
    

In [23]:
import numpy as np

In [31]:
for p in list(range(0,len(list_index_f))): ## loop para extraer y los datos para cada ubigeo
        ubigeos_des.append(df.
                           pipe(extrac_freq_ubigeo,p).
                           pipe(clean_columns_answers)
                          )
df_f=(pd.concat(ubigeos_des,axis=0))
df_f=df_f.rename({"P3A+: SABE LEER Y ESCRIBIR":"fila"},axis=1)

In [36]:
def multi_index(df):
    df=df.copy()
    df=pd.DataFrame(df.
                    rename({0:"freq"},axis=1).
                    set_index("fila").
                    stack())
    return df

def clean_data(df):
    df=df.copy()    
    df.index=(df.
              index.
              set_names(["ubigeo","fila","columna"])
             )
    df=(df.
        reset_index("columna").
        rename(index=str.strip).
        pipe(clean_str, column="columna").
        query('columna != "UBIGEO"').
        rename({0:'freq'},axis=1).
        replace("-","0",regex=True).
        assign(freq=lambda df_:df_.freq.astype(int))
       )
    return df
#.set_index([df.index,"columna"])

df_f.groupby('ubigeo').apply(multi_index)#.pipe(clean_data)

0
ubigeo fila                                           
0101   SI SABE LEER Y ESCRIBIR Urbano encuesta  32 036
                               Rural encuesta   14 747
                               Total            46 783
                               ubigeo             0101
       NO SABE LEER Y ESCRIBIR Urbano encuesta   3 391
...                                                ...
2504   SI SABE LEER Y ESCRIBIR ubigeo             2504
       NO SABE LEER Y ESCRIBIR Urbano encuesta       -
                               Rural encuesta      695
                               Total               695
                               ubigeo             2504

[4704 rows x 1 columns]

In [2]:
df1=merged.iloc[5:].copy() ## posible BUG
redapy.tabla_cruzada(df1)

TypeError: descriptor 'strip' for 'str' objects doesn't apply to a 'tuple' object

In [22]:
ruta_l=r"Bases originales"
df4=pd.read_excel(ruta_l+"\\ocupación_2017.xlsx") ## ocupaciones 2017

df5=pd.read_excel(ruta_l+"\\Edad en años_2017.xlsx") ## edad en años

In [27]:
df1=merged.iloc[5:].copy() ## posible BUG 
df1[1]=df1[1].astype(str)
redapy.conversion_redatam_matriz(df1,continuous=True, kind="descriptivos")
# redapy.conversion_redatam_matriz(df1)


AssertionError: La variable bajo análisis no es continua

In [21]:
df1.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
dtype: object

In [24]:
redapy.conversion_redatam_matriz(df5,continuous=True, kind="intervalos",intervalo=30)

fre                      
resp      0-29   30-59  60-89 90-119
ubigeo                              
0101     27105   18869   6132  167.0
0102     40394   23499   6403  148.0
0103     12510    9115   2932   92.0
0104     28737   10550   1604   10.0
0105     22843   14782   5900  201.0
...        ...     ...    ...    ...
2403     27282   16845   3651   71.0
2501    216663  125453  31943  572.0
2502     32783   13080   2167   27.0
2503     34806   19713   3389   46.0
2504      1722     881    154    0.0

[196 rows x 4 columns]